<a href="https://colab.research.google.com/github/bskarda/data-science-notebooks/blob/main/BERT_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 22.6 MB/s 
     |████████████████████████████████| 77 kB 1.1 MB/s 
     |████████████████████████████████| 6.6 MB 24.6 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentence-transformers  

     |████████████████████████████████| 79 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 15.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0aa777ece02d3a1d2fd76c3df8f04646bb97084018a3fc422e484e3af587b26a
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
!pip install datasets

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 45.3 MB/s 
     |████████████████████████████████| 1.1 MB 63.1 MB/s 
     |████████████████████████████████| 212 kB 60.6 MB/s 
     |████████████████████████████████| 127 kB 60.0 MB/s 
     |████████████████████████████████| 144 kB 55.4 MB/s 
     |████████████████████████████████| 271 kB 73.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


**Word prediction**

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("In-person and virtual events are both [MASK].")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

[{'score': 0.4464682638645172,
  'sequence': 'in - person and virtual events are both available.',
  'token': 2800,
  'token_str': 'available'},
 {'score': 0.14930403232574463,
  'sequence': 'in - person and virtual events are both possible.',
  'token': 2825,
  'token_str': 'possible'},
 {'score': 0.06877840310335159,
  'sequence': 'in - person and virtual events are both included.',
  'token': 2443,
  'token_str': 'included'},
 {'score': 0.05388854071497917,
  'sequence': 'in - person and virtual events are both offered.',
  'token': 3253,
  'token_str': 'offered'},
 {'score': 0.030450858175754547,
  'sequence': 'in - person and virtual events are both supported.',
  'token': 3569,
  'token_str': 'supported'}]

**Keyphrase extraction**

In [ ]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, candidates, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)
    
    #print('distances_candidates:', distances_candidates)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    #print('words_vals:', words_vals)
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            #print('candidate:', candidate)
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
#import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphrase
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
doc = """
         The AWS Jam ProServe offering allows you to run gamified events where participants experience a wide range of AWS services in a series of prepared challenges that are developed by AWS SMEs! There are currently over 80 public challenges within our Jam library, across themes of security, data analytics, ML AI, VR, migration tooling, and DevOps. This session will go over the process around developing challenges and then we will break out into teams to develop brand new challenges that highlight how AWS services can help our customers adopt the cloud faster. These challenges will then be used at our customer and public events going forward which will count toward your artifact goals!      """

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (0, 2)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, 
                        stop_words=stop_words
                        ).fit([doc])
candidates = count.get_feature_names_out()

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
#model = SentenceTransformer('roberta-large')

doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['library themes',
 'migration tooling',
 'new challenges',
 'vr migration',
 'data analytics']

In [ ]:
keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, 5, 20)
keywords

['80 public',
 'brand new',
 'faster challenges',
 'library themes',
 'data analytics']

In [ ]:
keywords = mmr(doc_embedding, candidate_embeddings, candidates, 5, 0.7)
keywords

['data analytics',
 'faster challenges',
 '80 public',
 'library themes',
 'help customers']

**Zero-shot classification: industry sector classification of session description text**

In [ ]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

sequence= ['With a new era of events on the horizon, Advancement leaders across higher education are taking what they’ve learned to expand the reach, impact, and ROI of their events programs across multiple event formats.',
'Don’t miss this perennial favorite Cvent CONNECT session where the Cvent Product Management team shares what’s new and what’s next for the Cvent Event Marketing and Management Platform. ',
'Industry Roadmap: The 2022 Event Landscape for Associations and Non-Profits',
]

#labels = ['Consumer Services', 'Technology', 'Finance', 'Education']
labels = ['Consumer Services', 'Technology', 'Finance', 'Health Care', 'Basic Industries', 'Consumer Durables', 'Consumer Non-durables', \
          'Education', 'Transportation', 'Government', 'Capital Goods', 'Public Utilities', 'Energy', 'Miscellaneous']
classifier(sequence, labels)

[{'labels': ['Education',
   'Capital Goods',
   'Energy',
   'Technology',
   'Miscellaneous',
   'Consumer Non-durables',
   'Consumer Durables',
   'Transportation',
   'Consumer Services',
   'Basic Industries',
   'Government',
   'Finance',
   'Public Utilities',
   'Health Care'],
  'scores': [0.7858461737632751,
   0.027380600571632385,
   0.027240470051765442,
   0.024874309077858925,
   0.024115849286317825,
   0.01976894959807396,
   0.01501650083810091,
   0.014865407720208168,
   0.012350286357104778,
   0.010981187224388123,
   0.010606415569782257,
   0.010040683671832085,
   0.009904366917908192,
   0.007009013555943966],
  'sequence': 'With a new era of events on the horizon, Advancement leaders across higher education are taking what they’ve learned to expand the reach, impact, and ROI of their events programs across multiple event formats.'},
 {'labels': ['Technology',
   'Capital Goods',
   'Consumer Non-durables',
   'Energy',
   'Education',
   'Consumer Services'

**Miscellaneous exploration**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [ ]:
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_train_dataset = raw_datasets["validation"]
raw_train_dataset[86]

{'idx': 796,
 'label': 1,
 'sentence1': 'He was arrested Friday night at an Alpharetta seafood restaurant while dining with his wife , singer Whitney Houston .',
 'sentence2': 'He was arrested again Friday night at an Alpharetta restaurant where he was having dinner with his wife .'}